In [1]:
import streamlit as st
import numpy as np
import pandas as pd
import numpy as np
import requests
import json
import os
import datetime
import dotenv 

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [8]:
r = requests.get('https://httpbin.org/user-agent')
useragent = json.loads(r.text)['user-agent']
headers = {'User-agent': useragent}

In [5]:
import googlemaps
from datetime import datetime, timedelta
import dotenv
import json
import pandas as pd

dotenv.load_dotenv()
api_key_google = os.getenv('api_key') 

gmaps = googlemaps.Client(key=api_key_google)

# Define the origin and destination
origin = "Richmond, VA"
destination = "Chantilly, VA"

# Define parameters for the directions request
params = {
    'origin': origin,
    'destination': destination,
    'mode': 'driving',  # You can change this to 'walking', 'bicycling', or 'transit'
    'alternatives': True,  # If you want alternative routes
    'avoid': None,  # You can specify features to avoid, e.g., 'tolls', 'highways'
    'language': 'en',  # Language for the results
    'units': 'imperial',  # Use 'metric' for metric units
    'departure_time': datetime.now(),  # Use the current time as the departure time
}

# Fetch directions
directions = gmaps.directions(**params)

# Check if directions were returned
if directions:
    # Print the total number of routes found
    print(f'Total routes found: {len(directions)}')
    
    for idx, route in enumerate(directions):
        print(f"\nRoute {idx + 1}:")
        print(f"  Distance: {route['legs'][0]['distance']['text']}")
        print(f"  Duration: {route['legs'][0]['duration']['text']}")
        print(f"  Start Address: {route['legs'][0]['start_address']}")
        print(f"  End Address: {route['legs'][0]['end_address']}")
        
        print("\n  Directions:")
        for step in route['legs'][0]['steps']:
            print(f"    - {step['html_instructions']} (Approx. {step['duration']['text']})")
else:
    print("No directions found.")

Total routes found: 3

Route 1:
  Distance: 108 mi
  Duration: 1 hour 51 mins
  Start Address: Richmond, VA, USA
  End Address: Chantilly, VA, USA

  Directions:
    - Head <b>northwest</b> on <b>E Grace St</b> toward <b>N 7th St</b> (Approx. 1 min)
    - Turn <b>right</b> at the 1st cross street onto <b>N 7th St</b> (Approx. 2 mins)
    - Slight <b>right</b> onto the <b>I-64 W</b>/<wbr/><b>I-95 N</b> ramp (Approx. 1 min)
    - Merge onto <b>I-95 N</b> (Approx. 4 mins)
    - Keep <b>left</b> to stay on <b>I-95 N</b> (Approx. 57 mins)
    - Continue onto <b>I-95 N</b> (Approx. 16 mins)
    - Take exit <b>160B</b> to merge onto <b>VA-123 N</b>/<wbr/><b>Gordon Blvd</b><div style="font-size:0.9em">Continue to follow VA-123 N</div><div style="font-size:0.9em">Pass by 7-Eleven (on the left in 1.1 mi)</div> (Approx. 16 mins)
    - Take the <b>VA-286 N</b>/<wbr/><b>Fairfax County Parkway</b> ramp (Approx. 1 min)
    - Merge onto <b>VA-286 N</b>/<wbr/><b>Fairfax County Pkwy</b> (Approx. 9 mins)

In [6]:
news_df = pd.json_normalize(directions[0], record_path=['legs', 'steps'])

news_df['time_in_minutes'] = round(news_df['duration.value']/60, 0)
news_df['time_in_minutes'] = news_df['time_in_minutes'].astype('int')
news_df['cumulative_time'] = news_df['time_in_minutes'].cumsum()

news_df['date_time'] = news_df['cumulative_time'].apply(lambda x: datetime.now() + timedelta(minutes=x))
news_df['date_time'] = news_df['date_time'].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))

max_time = news_df['cumulative_time'].iloc[(len(news_df)-1)]

desired_val = list(range(0,max_time+60, 60))
news_df1 = news_df[['end_location.lat', 'end_location.lng', 'cumulative_time', 'date_time']]
closest_rows = (news_df1.loc[[abs(news_df1['cumulative_time'] - hour).idxmin() for hour in desired_val]]).reset_index()

In [36]:
closest_rows['date_time'] = pd.to_datetime(closest_rows['date_time'])
closest_rows


,index,end_location.lat,end_location.lng,cumulative_time,date_time
0,0,37.541017,-77.436471,0,2024-10-14 18:00:00
1,4,38.466398,-77.406596,64,2024-10-14 19:00:00
2,10,38.894258,-77.431114,112,2024-10-14 20:00:00


In [41]:
temp_list = []
precip_chance_list = []
wind_speed_list = []
skycover_list = []
snowfall_list = []
windgust_list = []
heat_index_list = []
windchill_index_list = []

for i in range(len(closest_rows)):

    desired_val = [closest_rows['date_time'][i]]

    url = f"https://api.weather.gov/points/{closest_rows['end_location.lat'][i]},{closest_rows['end_location.lng'][i]}"
    r = requests.get(url, headers = headers)

    myjson = json.loads(r.text)
    df_url_info = pd.json_normalize(myjson['properties'])

    hourlyURL = df_url_info['forecastHourly'].iloc[0]   
    hourlyURL_grid = df_url_info['forecastGridData'].iloc[0] 

    # Obtain actual hourly forecast data

    r = requests.get(hourlyURL, headers = headers)
    r_g = requests.get(hourlyURL_grid, headers = headers)

    myjson = json.loads(r.text)
    myjson_g = json.loads(r_g.text)

    df = pd.json_normalize(myjson['properties']['periods'])
    df['startTime'] = pd.to_datetime(df['startTime']).dt.tz_localize(None)

    df = (df.loc[[abs(df['startTime'] - hour).idxmin() for hour in desired_val]]).reset_index()


    df_temp = df['temperature']
    df_pop = df['probabilityOfPrecipitation.value']
    df_ws = df['windSpeed']

    temp_list.append(df_temp)
    precip_chance_list.append(df_pop)
    wind_speed_list.append(df_ws)




    # df_sky = pd.json_normalize(myjson_g['properties']['skyCover']['values'])
    # df_wg = pd.json_normalize(myjson_g['properties']['windGust']['values'])
    # df_snow = pd.json_normalize(myjson_g['properties']['snowfallAmount']['values'])
    # df_HI = pd.json_normalize(myjson_g['properties']['heatIndex']['values'])
    # df_WC = pd.json_normalize(myjson_g['properties']['windChill']['values'])

    # def convert_time(var_name):

    #     var_name['validTime'] = var_name['validTime'].str.extract(r'^(.*?)/')
    #     var_name['validTime'] = pd.to_datetime(var_name['validTime'])
    #     return var_name

    # df_skycover = convert_time(df_sky)
    # df_windgust = convert_time(df_wg)
    # df_snowfall = convert_time(df_snow)
    # df_heat_index = convert_time(df_HI)
    # df_heat_index['value'] = (df_heat_index['value'] * 9/5) + 32
    # df_wind_chill = convert_time(df_WC)
    

In [42]:
temp_list

[0    56
 Name: temperature, dtype: int64,
 0    52
 Name: temperature, dtype: int64,
 0    51
 Name: temperature, dtype: int64]

In [25]:
url = f"https://api.weather.gov/points/{35},{-90}"
r = requests.get(url, headers = headers)

In [26]:
myjson = json.loads(r.text)
df_url_info = pd.json_normalize(myjson['properties'])

In [27]:
df_url_info

,@id,@type,cwa,forecastOffice,gridId,gridX,gridY,forecast,forecastHourly,forecastGridData,...,radarStation,relativeLocation.type,relativeLocation.geometry.type,relativeLocation.geometry.coordinates,relativeLocation.properties.city,relativeLocation.properties.state,relativeLocation.properties.distance.unitCode,relativeLocation.properties.distance.value,relativeLocation.properties.bearing.unitCode,relativeLocation.properties.bearing.value
0,"https://api.weather.gov/points/35,-90",wx:Point,MEG,https://api.weather.gov/offices/MEG,MEG,44,60,"https://api.weather.gov/gridpoints/MEG/44,60/f...","https://api.weather.gov/gridpoints/MEG/44,60/f...","https://api.weather.gov/gridpoints/MEG/44,60",...,KNQA,Feature,Point,"[-89.977669, 34.951005]",Southaven,MS,wmoUnit:m,5815.540383,wmoUnit:degree_(angle),339


In [28]:
hourlyURL = df_url_info['forecastHourly'].iloc[0]   
hourlyURL_grid = df_url_info['forecastGridData'].iloc[0]

In [29]:
hourlyURL

'https://api.weather.gov/gridpoints/MEG/44,60/forecast/hourly'

In [30]:
r = requests.get(hourlyURL, headers = headers)
r_g = requests.get(hourlyURL_grid, headers = headers)

myjson = json.loads(r.text)
myjson_g = json.loads(r_g.text)

df = pd.json_normalize(myjson['properties']['periods'])

In [31]:
df

,number,name,startTime,endTime,isDaytime,temperature,temperatureUnit,temperatureTrend,windSpeed,windDirection,icon,shortForecast,detailedForecast,probabilityOfPrecipitation.unitCode,probabilityOfPrecipitation.value,dewpoint.unitCode,dewpoint.value,relativeHumidity.unitCode,relativeHumidity.value
0,1,,2024-10-14T19:00:00-05:00,2024-10-14T20:00:00-05:00,False,62,F,,5 mph,N,https://api.weather.gov/icons/land/night/skc?s...,Clear,,wmoUnit:percent,0,wmoUnit:degC,2.777778,wmoUnit:percent,40
1,2,,2024-10-14T20:00:00-05:00,2024-10-14T21:00:00-05:00,False,58,F,,5 mph,NE,https://api.weather.gov/icons/land/night/skc?s...,Clear,,wmoUnit:percent,0,wmoUnit:degC,3.333333,wmoUnit:percent,47
2,3,,2024-10-14T21:00:00-05:00,2024-10-14T22:00:00-05:00,False,55,F,,5 mph,NE,https://api.weather.gov/icons/land/night/skc?s...,Clear,,wmoUnit:percent,0,wmoUnit:degC,3.333333,wmoUnit:percent,53
3,4,,2024-10-14T22:00:00-05:00,2024-10-14T23:00:00-05:00,False,54,F,,5 mph,ENE,https://api.weather.gov/icons/land/night/skc?s...,Clear,,wmoUnit:percent,0,wmoUnit:degC,3.333333,wmoUnit:percent,55
4,5,,2024-10-14T23:00:00-05:00,2024-10-15T00:00:00-05:00,False,52,F,,0 mph,,https://api.weather.gov/icons/land/night/skc?s...,Clear,,wmoUnit:percent,0,wmoUnit:degC,3.333333,wmoUnit:percent,59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,152,,2024-10-21T02:00:00-05:00,2024-10-21T03:00:00-05:00,False,54,F,,5 mph,E,https://api.weather.gov/icons/land/night/few?s...,Mostly Clear,,wmoUnit:percent,3,wmoUnit:degC,10.000000,wmoUnit:percent,84
152,153,,2024-10-21T03:00:00-05:00,2024-10-21T04:00:00-05:00,False,54,F,,5 mph,E,https://api.weather.gov/icons/land/night/few?s...,Mostly Clear,,wmoUnit:percent,3,wmoUnit:degC,9.444444,wmoUnit:percent,85
153,154,,2024-10-21T04:00:00-05:00,2024-10-21T05:00:00-05:00,False,53,F,,5 mph,E,https://api.weather.gov/icons/land/night/few?s...,Mostly Clear,,wmoUnit:percent,3,wmoUnit:degC,9.444444,wmoUnit:percent,86
154,155,,2024-10-21T05:00:00-05:00,2024-10-21T06:00:00-05:00,False,52,F,,5 mph,E,https://api.weather.gov/icons/land/night/few?s...,Mostly Clear,,wmoUnit:percent,3,wmoUnit:degC,9.444444,wmoUnit:percent,90


In [32]:
myjson_g

{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
  {'@version': '1.1',
   'wmoUnit': 'https://codes.wmo.int/common/unit/',
   'nwsUnit': 'https://api.weather.gov/ontology/unit/'}],
 'id': 'https://api.weather.gov/gridpoints/MEG/44,60',
 'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-90.0175633, 35.0039994],
    [-90.0185716, 34.9815299],
    [-89.99114420000001, 34.980701499999995],
    [-89.99013040000001, 35.00317089999999],
    [-90.0175633, 35.0039994]]]},
 'properties': {'@id': 'https://api.weather.gov/gridpoints/MEG/44,60',
  '@type': 'wx:Gridpoint',
  'updateTime': '2024-10-14T23:36:46+00:00',
  'validTimes': '2024-10-14T17:00:00+00:00/P7DT8H',
  'elevation': {'unitCode': 'wmoUnit:m', 'value': 95.0976},
  'forecastOffice': 'https://api.weather.gov/offices/MEG',
  'gridId': 'MEG',
  'gridX': '44',
  'gridY': '60',
  'temperature': {'uom': 'wmoUnit:degC',
   'values': [{'validTime': '2024-10-14T17:00:00+00:00/PT1H',
     'value': 17.